## Kütüphane Tanımları

In [7]:
import pandas as pd
import numpy as np
import string
import re # regular expressions
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from joblib import Parallel, delayed 
import joblib 

import utils

### Veri Ön İşlemleri
Elimizdeki verilerden gereksiz olan sütunu siliyoruz.

In [5]:
df = pd.read_csv("./datas/temizlenmemis_tweets.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

username                                              tweet
0  netflixturkiye                       @valenciaessek Mutlu yıllar!
1  netflixturkiye                          Yeni sezon şimdi yayında.
2  netflixturkiye             Geceye yeni yıldan bir beklenti bırak.
3  netflixturkiye  Özür dilerim ben de spoiler verdim, 20 puanla ...
4  netflixturkiye  2022 yılı hatalar testi:\n\nSpoiler verdim (20...

Modelin daha tutarlı olabilmesi için tweetler olabildiğince sadeleştirilmelidir. Bu nedenle bir anlam ifade etmeyen kelimeler tweetlerden ayıklanacaktır.

In [8]:
def clean(tweet):
    """
    Bu fonksiyon tweetlerin icindeki anlamsiz kelimeleri, noktalama isaretlerini ve rakamlari cikararak daha sade bir bicime donusturur.

    Parameters:
    tweet (str): Temizlenecek olan tweet

    Returns:
    row (str): Temizlenmis tweet
    """
    satirlar = tweet.replace("\n", " ") # Veri setinde \n yerine " " (bosluk) koyuluyor.
    satirlar = re.sub("[0-9]+", "", satirlar) # veri icerisindeki sayilar cikariliyor
    satirlar = [t for t in satirlar.split()if t not in utils.stop_words] # veri icerisindeki anlamsiz kelimeler cikariliyor
    
    return " ".join(satirlar).lower().translate(str.maketrans("", "", string.punctuation)) # veri icindeki noktalama isaretleri temizleniyor.

Tüm tweetlere temizleme ve sadeleştirme işlemi uygulanıyor.

In [9]:
df["clean"] = df.apply(lambda row: clean(row["tweet"]), axis = 1)
df

username                                              tweet  \
0      netflixturkiye                       @valenciaessek Mutlu yıllar!   
1      netflixturkiye                          Yeni sezon şimdi yayında.   
2      netflixturkiye             Geceye yeni yıldan bir beklenti bırak.   
3      netflixturkiye  Özür dilerim ben de spoiler verdim, 20 puanla ...   
4      netflixturkiye  2022 yılı hatalar testi:\n\nSpoiler verdim (20...   
...               ...                                                ...   
11460     RasimOzan_K  Bu şerefsizliğe alet olarak bu haysiyetsizce y...   
11461     RasimOzan_K  Aleni yalanlarla bana bu şerefsizliği yapan fi...   
11462     RasimOzan_K  Bu yalan açıklamaları yapan şahsa ve dolandırı...   
11463     RasimOzan_K  Ben asla böyle bir dilekçe vermedim.Hakkımda b...   
11464     RasimOzan_K  Benim bir mahkemede “Ben Erdoğan’ı destekliyor...   

                                                   clean  
0                             valenciaessek mutlu yıllar  
1                               yeni sezon şimdi yayında  
2                  geceye yeni yıldan bir beklenti bırak  
3      özür dilerim ben de spoiler verdim puanla kapa...  
4      yılı hatalar testi spoiler verdim  puan önerdi...  
...                                                  ...  
11460  bu şerefsizliğe alet olarak bu haysiyetsizce y...  
11461  aleni yalanlarla bana bu şerefsizliği yapan fi...  
11462  bu yalan açıklamaları yapan şahsa ve dolandırı...  
11463  ben asla böyle bir dilekçe vermedimhakkımda ba...  
11464  benim bir mahkemede “ben erdoğan’ı destekliyor...  

[11465 rows x 3 columns]

In [10]:
# Veri setinde yer alan kullanıcılar
users = list(df["username"].unique())
users

['netflixturkiye', 'webtekno', 'acunilicali', 'RasimOzan_K']

In [11]:
def etiketle(row, users):
    """
    Veri setine Label Encoding islemi yapar.

    Parameters:
    row (str): Label Encode edilecek satir
    users (list): Label Encode islemi yapilirken referans alinacak olan, veri setinde bulunan kullanicilarin listesi.

    Returns:
    (int)
    """
    if row["username"] == users[0]:
        return 0 
    elif row["username"] == users[1]:
        return 1
    elif row["username"] == users[2]:
        return 2
    return 3

In [12]:
# Veri setine Label Encoding islemi yapiliyor
df["labels"] = df.apply(lambda row:  etiketle(row, users), axis=1)
df

username                                              tweet  \
0      netflixturkiye                       @valenciaessek Mutlu yıllar!   
1      netflixturkiye                          Yeni sezon şimdi yayında.   
2      netflixturkiye             Geceye yeni yıldan bir beklenti bırak.   
3      netflixturkiye  Özür dilerim ben de spoiler verdim, 20 puanla ...   
4      netflixturkiye  2022 yılı hatalar testi:\n\nSpoiler verdim (20...   
...               ...                                                ...   
11460     RasimOzan_K  Bu şerefsizliğe alet olarak bu haysiyetsizce y...   
11461     RasimOzan_K  Aleni yalanlarla bana bu şerefsizliği yapan fi...   
11462     RasimOzan_K  Bu yalan açıklamaları yapan şahsa ve dolandırı...   
11463     RasimOzan_K  Ben asla böyle bir dilekçe vermedim.Hakkımda b...   
11464     RasimOzan_K  Benim bir mahkemede “Ben Erdoğan’ı destekliyor...   

                                                   clean  labels  
0                             valenciaessek mutlu yıllar       0  
1                               yeni sezon şimdi yayında       0  
2                  geceye yeni yıldan bir beklenti bırak       0  
3      özür dilerim ben de spoiler verdim puanla kapa...       0  
4      yılı hatalar testi spoiler verdim  puan önerdi...       0  
...                                                  ...     ...  
11460  bu şerefsizliğe alet olarak bu haysiyetsizce y...       3  
11461  aleni yalanlarla bana bu şerefsizliği yapan fi...       3  
11462  bu yalan açıklamaları yapan şahsa ve dolandırı...       3  
11463  ben asla böyle bir dilekçe vermedimhakkımda ba...       3  
11464  benim bir mahkemede “ben erdoğan’ı destekliyor...       3  

[11465 rows x 4 columns]

## Veri Setinin Modele Uygun Hazirlanmasi

In [13]:
X = df.clean.to_numpy() # Temizlenmis tweetler numpy arraye donusturuluyor
y = df.labels.to_numpy() # Tweetlerin etiketleri numpy arraye donusturuluyor

# Veri seti %80 egitim, %20 test verisi olacak sekilde ayiriliyor.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF yontemiyle tweetler vektorlere donusturuluyor
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Multinomial Naive Bayes Nedir?
Multinomial Naive Bayes (MultinomialNB), Bayes teoremi üzerine kurulmuş ve belirli varsayımlara dayanan olasılıksal bir sınıflandırma algoritmasıdır. Bu algoritma, özellikle kelime sayımlarına dayalı metin sınıflandırma problemleri için uygundur. Naive Bayes sınıflandırıcıları, basit ama güçlü ve hızlı algoritmalardır.

In [14]:
# Multinomial Naive Bayes modelinde egitim yapiliyor
model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

# Modelin veri seti uzerindeki basarisi
print("NB train accuracy:", model_NB.score(X_train, y_train))
print("NB test accuracy:", model_NB.score(X_test, y_test))

NB train accuracy: 0.9615133013519407
NB test accuracy: 0.8634976013955517


### Decision Tree Classifier Nedir?
Siniflandirma problemleri icin kullanilan bir karar agaci siniflandiricisidir. Karar agaclari, veri kümesindeki ornekleri ozelliklerine gore dallara ayirarak siniflandirma islemi yapar.

In [15]:
# Decision Tree Classifier modelinde egitim yapiliyor
model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)

# Modelin veri seti uzerindeki basarisi
print("DT train accuracy:", model_DT.score(X_train, y_train))
print("DT test accuracy:", model_DT.score(X_test, y_test))

DT train accuracy: 0.9996729175752289
DT test accuracy: 0.7056258177060619


## Degerlendirme
Naive Bayes ve Decision Tree algoritmalarıyla egitilen modellerin test veri seti userindeki basarilari sirasiyla %86 ve %70 olarak olculmustur.

## Test

In [16]:
tweet = clean("İzlediğin son filmi söyle, nasıl hissettiğini tahmin edeyim.")
tweet_vec = vectorizer.transform([tweet])
# Netflix: 0
# Webtekno: 1
# Acun: 2
# ROK: 3
print("NB:", model_NB.predict(tweet_vec))
print("DT:", model_DT.predict(tweet_vec))

result_NB = model_NB.predict_proba(tweet_vec)
result_DT = model_DT.predict_proba(tweet_vec)
print(
    f"Netflix: \tNB: {round(100*result_NB[0][0],2)} \tDT: {round(100*result_DT[0][0],2)}\n"
    f"Webtekno: \tNB: {round(100*result_NB[0][1],2)} \tDT: {round(100*result_DT[0][1],2)}\n"
    f"Acun Ilicali: \tNB: {round(100*result_NB[0][2],2)} \tDT: {round(100*result_DT[0][2],2)}\n"
    f"Rasim Ozan Kutahyali: \tNB: {round(100*result_NB[0][3],2)} \tDT{round(100*result_DT[0][3],2)}"
    )

NB: [0]
DT: [1]
Netflix: 	NB: 77.82 	DT: 0.0
Webtekno: 	NB: 17.24 	DT: 100.0
Acun Ilicali: 	NB: 0.27 	DT: 0.0
Rasim Ozan Kutahyali: 	NB: 4.66 	DT0.0


In [17]:
def tweet_analyse(tweet):
    tweet = clean(tweet)
    tweet_vec = vectorizer.transform([tweet])
    # Netflix: 0
    # Webtekno: 1
    # Acun: 2
    # ROK: 3
    print("NB:", model_NB.predict(tweet_vec))
    print("DT:", model_DT.predict(tweet_vec))
    
    result_NB = model_NB.predict_proba(tweet_vec)
    result_DT = model_DT.predict_proba(tweet_vec)
    print(
        f"Netflix: \tNB: {round(100*result_NB[0][0],2)} \tDT: {round(100*result_DT[0][0],2)}\n"
        f"Webtekno: \tNB: {round(100*result_NB[0][1],2)} \tDT: {round(100*result_DT[0][1],2)}\n"
        f"Acun Ilicali: \tNB: {round(100*result_NB[0][2],2)} \tDT: {round(100*result_DT[0][2],2)}\n"
        f"Rasim Ozan Kutahyali: \tNB: {round(100*result_NB[0][3],2)} \tDT{round(100*result_DT[0][3],2)}"
        )

In [18]:
tweet_analyse("İzlediğin son filmi söyle, nasıl hissettiğini tahmin edeyim.")

NB: [0]
DT: [1]
Netflix: 	NB: 77.82 	DT: 0.0
Webtekno: 	NB: 17.24 	DT: 100.0
Acun Ilicali: 	NB: 0.27 	DT: 0.0
Rasim Ozan Kutahyali: 	NB: 4.66 	DT0.0


### Modelleri Kaydet

In [19]:
# TF-IDF vectorizer kaydet
joblib.dump(vectorizer, "./models/vectorizer.pkl")

['./models/vectorizer.pkl']

In [20]:
# Modelleri kaydet
joblib.dump(model_NB, './models/model_NB.pkl')
joblib.dump(model_DT, './models/model_DT.pkl')

['./models/model_DT.pkl']